# Water Mass Transformation diagnostics using ACCESS-OM2 neutral diagnostics

This is a example code for working with the Water Mass Transformation output from the neutral `_on_nrho` diagnostics for a tracer

In [7]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import os

import IPython.display

import cosima_cookbook as cc

At the moment the diagnostics have to be output from a particular run or runs of ACCESS-OM2. Specify the output directory on Gadi

In [59]:
output_dir = r'/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/'

In [60]:
outputs = sorted([_ for _ in os.listdir(output_dir) if _.startswith('output')])
dir_list = sorted([os.path.join(output_dir, _) for _ in outputs])
print(outputs)
print(dir_list)

['output050', 'output051', 'output052', 'output053', 'output054', 'output055']
['/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/output050', '/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/output051', '/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/output052', '/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/output053', '/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/output054', '/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/output055']


To make the database, specify a location you have write access to (assign `db_path`)

In [61]:
db_path = r'/g/data/e14/cb2411/data'
session = cc.database.create_session(os.path.join(db_path, 'wmt_diags.db'))

In [62]:
cc.database.build_index(dir_list,session,update=True)

  0%|          | 0/125 [00:00<?, ?it/s]

Indexing experiment: output050


 10%|█         | 2/20 [00:00<00:01, 13.96it/s]

Indexing experiment: output051


 10%|█         | 2/20 [00:00<00:01, 13.98it/s]

...

Indexing experiment: output054


  0%|          | 0/20 [00:00<?, ?it/s]

Indexing experiment: output055


100%|██████████| 20/20 [00:01<00:00, 17.08it/s]


225

In [94]:
# make list of experiments - at the moment this is the number of runs in a single experiment
# chat to COSIMA about making the cc.database.create_session(...) work with a dictionary
# to be able to specficy the file structure of the experiments 
# (running multiple "experiments" - say a control and perturbation will have the same names?)
exp = list(cc.querying.get_experiments(session)['experiment'])
exp

['output050', 'output051', 'output052', 'output053', 'output054', 'output055']

Specificy the neutral coordinate (either temperature or salt). This is use to construct the variable names used in the `.getvar` method

In [66]:
neutral_coordinate = 'salt'

In [97]:
diags = {'diffusion': ['salt_vdiffuse_diff_cbt', 'salt_nonlocal_KPP', 
                      'neutral_diffusion_salt','salt_vdiffuse_k33', 
                      'mixdownslope_salt','salt_sigma_diff'], 
        'skew-diffusion': ['neutral_gm_salt', 'salt_submeso'],
        'tracer_sbc': ['salt_vdiffuse_sbc', 'salt_eta_smooth', 'salt_rivermix'],
        'volume_sbc': ['mass_pmepr'],
        }